# CustomLayer 定義

如果你想要自己根據論文公式刻出一個model，下面一有一個最基本的範例可供使用
PyTorch 在定義model會需要用到Parameter來打包可訓練參數
這樣在優化器指定要優化的參數時，才可以透過Module.parameter()這個函式來快速調參

而要注意的是當資料是以Batch的形式呈現，計算時要注意batch的軸不能消失
否則在訓練資料的過程中就會失去Batch Training的意義

也就是說公式需要改寫為
## $$y=xW^T+b$$

In [43]:
import torch

In [44]:
# 自定義一個layer來做影像辨識
class LinearLayer(torch.nn.Module):

    def __init__(self, in_feature, out_feature) -> None:
        super(LinearLayer, self).__init__()
        self.w1 = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty(size = (out_feature, in_feature), requires_grad=True)))
        self.b1 = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty(size = (1, out_feature))).squeeze(0))
        
    def forward(self, x):
        x = x @ self.w1.T + self.b1
        return x

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [46]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [47]:
train_dataset = MNIST(root='./basic_dataset', train = True, download= True, transform=ToTensor())
test_dataset = MNIST(root='./basic_dataset', train = False, download= True, transform=ToTensor())

In [48]:
trainLoader = DataLoader(train_dataset, batch_size=32, shuffle= True)
testLoader = DataLoader(test_dataset, batch_size=32, shuffle= True)

In [49]:
import torch.nn as nn
import torch.optim as optim

In [50]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            LinearLayer(28*28, 512),
            nn.ReLU(),
            LinearLayer(512, 512),
            nn.ReLU(),
            LinearLayer(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): LinearLayer()
    (1): ReLU()
    (2): LinearLayer()
    (3): ReLU()
    (4): LinearLayer()
  )
)


In [51]:
for parm in model.parameters():
    print(parm)

Parameter containing:
tensor([[ 0.0514,  0.0014,  0.0230,  ...,  0.0055,  0.0263, -0.0310],
        [-0.0379,  0.0376, -0.0229,  ...,  0.0488,  0.0039, -0.0314],
        [ 0.0499,  0.0609, -0.0008,  ..., -0.0174,  0.0624, -0.0014],
        ...,
        [ 0.0024,  0.0220, -0.0107,  ...,  0.0538,  0.0397,  0.0126],
        [-0.0637, -0.0604, -0.0328,  ...,  0.0672, -0.0631,  0.0134],
        [ 0.0256,  0.0408,  0.0445,  ..., -0.0035,  0.0662,  0.0348]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 2.0173e-02, -1.9361e-02, -8.1276e-02,  8.9001e-02,  2.9033e-02,
        -5.1355e-02,  2.3378e-02, -6.9840e-02,  8.6917e-03, -1.9112e-02,
         6.6321e-02, -7.4989e-02, -5.0029e-02, -1.9164e-02,  7.4624e-02,
         4.5056e-02,  8.3704e-02, -8.3460e-02,  3.4388e-02,  3.0743e-02,
         1.0273e-01,  9.7634e-02,  6.7447e-02, -7.4686e-02,  5.0398e-02,
         3.5076e-02,  3.1794e-02,  1.6450e-02, -6.7021e-02, -3.5462e-02,
        -8.8568e-02, -9.9488e-02,  9.541

In [52]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [53]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [54]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [55]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainLoader, model, loss_fn, optimizer)
    test(testLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.442665  [    0/60000]
loss: 2.236519  [ 3200/60000]
loss: 2.206225  [ 6400/60000]
loss: 2.168396  [ 9600/60000]
loss: 2.124247  [12800/60000]
loss: 1.971074  [16000/60000]
loss: 1.983765  [19200/60000]
loss: 1.875924  [22400/60000]
loss: 1.853083  [25600/60000]
loss: 1.666651  [28800/60000]
loss: 1.629444  [32000/60000]
loss: 1.580033  [35200/60000]
loss: 1.502506  [38400/60000]
loss: 1.510396  [41600/60000]
loss: 1.278157  [44800/60000]
loss: 1.287291  [48000/60000]
loss: 1.135813  [51200/60000]
loss: 1.258750  [54400/60000]
loss: 1.095076  [57600/60000]
Test Error: 
 Accuracy: 81.6%, Avg loss: 1.074352 

Epoch 2
-------------------------------
loss: 0.992744  [    0/60000]
loss: 0.968783  [ 3200/60000]
loss: 0.944089  [ 6400/60000]
loss: 1.056798  [ 9600/60000]
loss: 1.049953  [12800/60000]
loss: 1.074222  [16000/60000]
loss: 0.626335  [19200/60000]
loss: 0.670370  [22400/60000]
loss: 0.833117  [25600/60000]
loss: 0.715325  [28800/60000